In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('florida_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_50_mean', 'new_cases_10_std', 'minimum_temperature_celsius', 'relative_humidity', 'maximum_temperature_celsius', 'new_cases_50_std', 'new_cases_100_mean', 'Booster_Doses_50Plus', 'average_temperature_celsius', 'current_intensive_care_patients', 'Booster_Doses', 'rainfall_mm', 'booster_rate', 'Series_Complete_5Plus', 'new_cases_100_max', 'new_cases_10_max', 'Booster_Doses_65Plus', 'new_cases_50_max', 'Booster_Doses_18Plus', 'new_deaths_100_std', 'dew_point', 'new_cases_50_min', 'vaccination_rate_1dose', 'Series_Complete_65Plus', 'new_cases_100_std', 'new_deaths_100_mean', 'Administered_Dose1_Recip', 'new_deaths_10_mean']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_cases_50_std', 'dew_point', 'new_cases_10_mean', 'minimum_temperature_celsius', 'new_hospitalized_patients', 'relative_humidity', 'current_intensive_care_patients', 'PEOPLE_DEATH_NEW_COUNT', 'average_temperature_celsius', 'new_cases_10_max', 'rainfall_mm', 'maximum_temperature_celsius', 'new_cases_50_mean', 'new_deaths_10_std', 'new_cases_100_std', 'new_deaths_100_std', 'new_deaths_10_mean', 'new_cases_10_min', 'new_deaths_50_mean', 'Administered_Dose1_Recip', 'new_deaths_50_std', 'Administered_Dose1_Recip_5Plus', 'new_deaths_100_mean', 'new_cases_100_mean', 'Booster_Doses_65Plus', 'Series_Complete_5Plus', 'Booster_Doses', 'Booster_Doses_50Plus', 'new_deaths_10_max']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'Series_Complete_Yes', 'new_cases_50_mean', 'relative_humidity', 'Booster_Doses', 'Administered_Dose1_Recip_18Plus', 'Administered_Dose1_Recip', 'Administered_Dose1_Recip_5Plus', 'maximum_temperature_celsius', 'current_intensive_care_patients', 'vaccination_rate_complete', 'minimum_temperature_celsius', 'new_cases_10_std', 'new_cases_50_std', 'new_deaths_100_mean', 'Series_Complete_5Plus', 'new_cases_50_min', 'Booster_Doses_50Plus', 'dew_point', 'new_deaths_50_mean', 'Series_Complete_12Plus', 'new_cases_50_max', 'Administered_Dose1_Recip_12Plus', 'new_cases_10_min', 'new_cases_10_max', 'new_cases_100_mean', 'new_deaths_50_max', 'Booster_Doses_65Plus', 'Booster_Doses_18Plus']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'Booster_Doses_65Plus', 'new_cases_50_mean', 'vaccination_rate_1dose', 'new_cases_10_std', 'new_cases_50_std', 'dew_point', 'Series_Complete_12Plus', 'relative_humidity', 'Series_Complete_5Plus', 'Administered_Dose1_Recip_12Plus', 'new_cases_50_max', 'new_cases_100_max', 'Administered_Dose1_Recip', 'minimum_temperature_celsius', 'maximum_temperature_celsius', 'Series_Complete_65Plus', 'Booster_Doses_50Plus', 'Booster_Doses_18Plus', 'new_cases_10_max', 'current_intensive_care_patients', 'Series_Complete_Yes', 'average_temperature_celsius', 'new_cases_100_mean', 'vaccination_rate_complete', 'new_cases_100_std', 'rainfall_mm', 'new_deaths_100_mean', 'new_deaths_100_std']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_cases_50_mean', 'new_hospitalized_patients', 'new_cases_10_std', 'current_intensive_care_patients', 'relative_humidity', 'Series_Complete_12Plus', 'Series_Complete_18Plus', 'maximum_temperature_celsius', 'Administered_Dose1_Recip_5Plus', 'Administered_Dose1_Recip_18Plus', 'minimum_temperature_celsius', 'Booster_Doses_18Plus', 'new_cases_50_max', 'new_cases_50_std', 'dew_point', 'Administered_Dose1_Recip', 'Series_Complete_5Plus', 'average_temperature_celsius', 'booster_rate', 'new_cases_10_max', 'Administered_Dose1_Recip_12Plus', 'new_cases_100_mean', 'Series_Complete_Yes', 'rainfall_mm', 'vaccination_rate_1dose', 'Administered_Dose1_Recip_65Plus', 'Booster_Doses_65Plus', 'vaccination_rate_complete', 'new_cases_50_min']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
----------------------------------------------------------------
Basis Function                             Pruned  Coefficient  
----------------------------------------------------------------
(Intercept)                                No      -844579      
h(new_cases_10_mean-24547.6)               Yes     None         
h(24547.6-new_cases_10_mean)               No      -3.85168     
h(new_cases_10_max-151760)                 Yes     None         
h(151760-new_cases_10_max)                 No      0.929899     
h(new_cases_50_mean-18042.2)               No      78.9968      
h(18042.2-new_cases_50_mean)               No      -12.3292     
h(new_cases_50_std-46157.3)                No      -79.6616     
h(46157.3-new_cases_50_std)                Yes     None         
h(new_hospitalized_patients-2092)          Yes     None         
h(2092-new_hospitalized_patients)          No      160.771      
h(current_intensive_care_patients-2057)    Yes     None         
h(2057-curren

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_cases_10_std', 'Series_Complete_5Plus', 'new_cases_10_mean', 'new_cases_50_std', 'current_intensive_care_patients', 'minimum_temperature_celsius', 'relative_humidity', 'dew_point', 'Booster_Doses_65Plus', 'maximum_temperature_celsius', 'new_cases_50_mean', 'new_cases_100_mean', 'new_cases_10_max', 'Administered_Dose1_Recip', 'new_hospitalized_patients'}


## XGBoost Feature Selection, includes lag and rolling

In [14]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [15]:
covid_df = pd.read_csv('florida_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [16]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_7_lag', 'new_deaths_50_mean', 'new_cases_10_max', 'new_deaths_10_std', 'new_cases_10_mean', 'new_cases_10_std', 'new_deaths_100_mean', 'Booster_Doses_18Plus', 'new_deaths_10_max', 'Administered_Dose1_Recip_18Plus', 'booster_rate', 'new_deaths_10_mean', 'new_hospitalized_patients', 'Series_Complete_5Plus', 'Booster_Doses_65Plus', 'Booster_Doses', 'Series_Complete_12Plus', 'new_cases_50_max', 'new_cases_50_lag', 'new_cases_1 lag', 'new_cases_100_mean', 'new_cases_100_max', 'new_deaths_3_lag', 'new_deaths_2_lag', 'Series_Complete_18Plus', 'new_cases_350_lag', 'maximum_temperature_celsius', 'Booster_Doses_50Plus', 'relative_humidity', 'new_cases_50_std', 'new_deaths_50_std', 'Administered_Dose1_Recip_5Plus', 'new_cases_10_min', 'new_cases_50_mean', 'Administered_Dose1_Recip_65Plus', 'new_deaths_100_max', 'rainfall_mm', 'new_deaths_50_lag', 'new_deaths_4_lag', 'PEOPLE_DEATH_NEW_COUNT']


In [17]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_10_std', 'new_cases_7_lag', 'new_cases_10_mean', 'new_cases_10_max', 'new_cases_3_lag', 'new_cases_1 lag', 'new_hospitalized_patients', 'new_cases_2_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_deaths_50_lag', 'new_cases_50_std', 'new_deaths_3_lag', 'maximum_temperature_celsius', 'relative_humidity', 'new_cases_350_lag', 'new_cases_4_lag', 'new_deaths_7_lag', 'current_intensive_care_patients', 'new_deaths_10_std', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_100_std', 'new_deaths_30_lag', 'rainfall_mm', 'dew_point', 'new_cases_150_lag', 'average_temperature_celsius', 'new_deaths_2_lag', 'new_deaths_1_lag', 'new_cases_10_min']


In [18]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_7_lag', 'Administered_Dose1_Recip', 'new_cases_10_max', 'new_cases_10_std', 'new_hospitalized_patients', 'new_cases_10_mean', 'facial_coverings', 'new_deaths_100_max', 'new_cases_10_min', 'new_cases_50_std', 'new_cases_1 lag', 'new_cases_2_lag', 'new_deaths_50_std', 'new_cases_3_lag', 'new_deaths_10_std', 'new_cases_50_lag', 'PEOPLE_DEATH_NEW_COUNT', 'dew_point', 'new_cases_100_min', 'new_deaths_3_lag', 'new_cases_350_lag', 'stringency_index', 'new_deaths_100_std', 'new_cases_100_std', 'new_deaths_50_lag', 'new_cases_150_lag', 'current_intensive_care_patients', 'new_deaths_1_lag', 'new_cases_50_mean', 'new_deaths_10_mean']


In [19]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
----------------------------------------------------------------------------
Basis Function                                         Pruned  Coefficient  
----------------------------------------------------------------------------
(Intercept)                                            No      10244.2      
present(new_cases_7_lag)                               Yes     None         
missing(new_cases_7_lag)                               Yes     None         
new_cases_7_lag*present(new_cases_7_lag)               No      1.08079      
h(new_cases_10_max-151760)                             No      1.00469      
h(151760-new_cases_10_max)                             No      -0.119841    
h(new_cases_50_mean-18042.2)                           No      -31.9169     
h(18042.2-new_cases_50_mean)                           No      0.342476     
h(current_intensive_care_patients-2521)                No      -9.28149     
h(2521-current_intensive_care_patients)                No      2

In [20]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [21]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [22]:
full = common_member(a, b, c)
full

{'new_cases_10_std', 'new_cases_350_lag', 'new_deaths_50_lag', 'new_hospitalized_patients', 'new_cases_50_std', 'new_cases_50_lag', 'new_cases_7_lag', 'new_deaths_3_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_min', 'new_deaths_10_std', 'new_cases_1 lag', 'new_cases_10_max', 'new_cases_10_mean'}
